<a href="https://colab.research.google.com/github/u6yuvi/DL-POC/blob/main/NLP/BERT_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pytorch-pretrained-bert
!pip install seqeval
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
import json
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras.models import Sequential
from keras.callbacks import CSVLogger
import pandas as pd
from keras_contrib.utils import save_load_utils
import matplotlib.pyplot as plt
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import nltk

In [ ]:
# Reading required Files
list_of_sent_tags = list()
list_of_sent_POS_tags = list()
list_of_sent_words = list()

with open("/content/drive/My Drive/Colab Notebooks/Corpus_Sentences_BERT.csv","r", encoding="utf8") as csv_file:
    csv_file = csv.reader(csv_file, delimiter='|', quotechar = '"')
    for row in csv_file:
        list_of_sent_words.append(row)

with open("/content/drive/My Drive/Colab Notebooks/Corpus_Sentences_Tags_BERT.csv","r", encoding="utf8") as csv_file:
    csv_file = csv.reader(csv_file, delimiter='|', quotechar = '"')
    for row in csv_file:
        list_of_sent_tags.append(row)
    
with open("/content/drive/My Drive/Colab Notebooks/Corpus_POS_Tags_BERT.csv","r", encoding="utf8") as csv_file:
    csv_file = csv.reader(csv_file, delimiter='|', quotechar = '"')
    for row in csv_file:
        list_of_sent_POS_tags.append(row)
		
list_of_sent_tags_test = list()
list_of_sent_POS_tags_test = list()
list_of_sent_words_test = list()

with open("/content/drive/My Drive/Colab Notebooks/Corpus_Sentences_Test_BERT.csv","r", encoding="utf8") as csv_file:
    csv_file = csv.reader(csv_file, delimiter='|', quotechar = '"')
    for row in csv_file:
        list_of_sent_words_test.append(row)

with open("/content/drive/My Drive/Colab Notebooks/Corpus_Sentences_Tags_Test_BERT.csv","r", encoding="utf8") as csv_file:
    csv_file = csv.reader(csv_file, delimiter='|', quotechar = '"')
    for row in csv_file:
        list_of_sent_tags_test.append(row)
    
with open("/content/drive/My Drive/Colab Notebooks/Corpus_POS_Tags_Test_BERT.csv","r", encoding="utf8") as csv_file:
    csv_file = csv.reader(csv_file, delimiter='|', quotechar = '"')
    for row in csv_file:
        list_of_sent_POS_tags_test.append(row)
		
new_sent_tags = list_of_sent_tags + list_of_sent_tags_test
new_sent_words = list_of_sent_words + list_of_sent_words_test
new_sent_POS_tags = list_of_sent_POS_tags + list_of_sent_POS_tags_test

req_list_test = [["Sentence: " + str(i + 1), word, POS, tag] for i, (sent, POS_sent, tag_sent) in enumerate(zip(list_of_sent_words_test, list_of_sent_POS_tags_test, list_of_sent_tags_test)) for word, POS, tag in zip(sent, POS_sent, tag_sent)]
data_test = pd.DataFrame(req_list_test, columns = ["Sentence #", "Word", "POS", "Tag"])
data_test.tail(20)

req_list_train = [["Sentence: " + str(i + 1), word, POS, tag] for i, (sent, POS_sent, tag_sent) in enumerate(zip(list_of_sent_words, list_of_sent_POS_tags, list_of_sent_tags)) for word, POS, tag in zip(sent, POS_sent, tag_sent)]
data_train = pd.DataFrame(req_list_train, columns = ["Sentence #", "Word", "POS", "Tag"])
data_train.tail(20)

req_list = [["Sentence: " + str(i + 1), word, POS, tag] for i, (sent, POS_sent, tag_sent) in enumerate(zip(new_sent_words, new_sent_POS_tags, new_sent_tags)) for word, POS, tag in zip(sent, POS_sent, tag_sent)]
data = pd.DataFrame(req_list, columns = ["Sentence #", "Word", "POS", "Tag"])


In [ ]:
words = list(set(data["Word"].values))
words = sorted(words)
words.append("endpad")
n_words = len(words); n_words

words_tr = list(set(data_train["Word"].values))
words_tr = sorted(words_tr)
words_tr.append("endpad")
n_words_tr = len(words_tr); n_words_tr

words_te = list(set(data_test["Word"].values))
words_te = sorted(words_te)
words_te.append("endpad")
n_words_te = len(words_te); n_words_te

tags_test = list(set(data_test["Tag"].values))
tags_test = sorted(tags_test)
n_tags_test = len(tags_test); n_tags_test

tags = list(set(data["Tag"].values))
tags = sorted(tags)
n_tags = len(tags); n_tags

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(data_train)
getter_test = SentenceGetter(data_test)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences_test = [" ".join([s[0] for s in sent]) for sent in getter_test.sentences]

tokenized_texts = [[s[0] for s in sent] for sent in getter.sentences]
tokenized_texts_test = [[s[0] for s in sent] for sent in getter_test.sentences]

labels = [[s[2] for s in sent] for sent in getter.sentences]
labels_test = [[s[2] for s in sent] for sent in getter_test.sentences]
print(labels[0])


In [ ]:
tags_vals = list(set(data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert import BertAdam
from pytorch_pretrained_bert import BertForTokenClassification

In [ ]:
MAX_LEN = 75
bs = 32

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0) 

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt[:512]) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt[:512]) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
tags_test = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_test],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
attention_masks_test = [[float(i>0) for i in ii] for ii in input_ids_test]

In [ ]:
"""
The BERT Model requires us to have a [SEP] token at the end of each sentence as a part of its preprocessing. 102 is the index BERT recognizes as the index of [SEP]. Hence, I am adding it to the end of the sentence after padding/truncating
(as it might have been removed if the sequences were greater than 75 in length) to be compatible with BERT's requirement. I didn't have it in the beginning and I thought it would be the reason for the poor results but changing it didn't help and I chose to keep it anyways as it felt right. :)
"""
for i, inp in enumerate(input_ids):
  if(102 not in inp):
    inp[-1] = 102
    tags[i][-1] = tag2idx.get("O")

for i, inp in enumerate(input_ids_test):
  if(102 not in inp):
    inp[-1] = 102
    tags_test[i][-1] = tag2idx.get("O")

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=10, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=10, test_size=0.1)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)

tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)

tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [ ]:
model.cuda();

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
from tqdm import tqdm, trange
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("\nTrain loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(valid_tags, pred_tags)))

In [ ]:
te_inputs = torch.tensor(input_ids_test)
te_tags = torch.tensor(tags_test)
te_masks = torch.tensor(attention_masks_test)

test_data = TensorDataset(te_inputs, te_masks, te_tags)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
test_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]

In [ ]:
tags_test_fin = list()
for l in tags_test:
  temp_tag = list()
  for l_i in l:
    temp_tag.append(tags_vals[l_i])
  tags_test_fin.append(temp_tag)

In [ ]:
for pred_label, test_label, label_test in zip(pred_tags, test_tags,tags_test_fin):
  for i, pl in enumerate(pred_label):
    if pl == 'X':
        pred_label[i] = pred_label[i-1].replace("B", "I")
  for i, tl in enumerate(test_label):
    if tl == 'X':
        test_label[i] = test_label[i-1].replace("B", "I")
  for i, tl in enumerate(label_test):
    if tl == 'X':
        label_test[i] = label_test[i-1].replace("B", "I")

In [ ]:
print("Test loss: {}".format(eval_loss/nb_eval_steps))
print("Test Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Test F1-Score: {}".format(f1_score(tags_test_fin, pred_tags)))

In [ ]:
print(classification_report(tags_test_fin, pred_tags))

In [ ]:
print("Number of Test sentences: ", len(tags_test_fin))
print("Number of Predicted sentences: ", len(train_data))